In [ ]:
# STEP 1: Install packages
!pip install transformers wikipedia colorama requests beautifulsoup4 pycountry geopy forex-python --quiet

# STEP 2: Import libraries
from transformers import pipeline
import wikipedia
import textwrap
import random
import time
from colorama import Fore, Style, Back, init
import requests
from bs4 import BeautifulSoup
import datetime
import pycountry
from geopy.geocoders import Nominatim
from forex_python.converter import CurrencyRates
import json

init(autoreset=True)

# Load summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
wikipedia.set_lang("en")

# Enhanced travel tips
travel_tips = [
    "📦 Pack light and bring only essentials - remember the 'half the clothes, twice the money' rule!",
    "🌐 Download offline maps and translation apps before your trip.",
    "🌅 Visit major attractions during early morning or late afternoon to avoid crowds.",
    "🚖 Research public transportation passes that can save you money over individual tickets.",
    "📱 Buy a local SIM card or eSIM for cheap data access and avoid roaming charges.",
    "🔌 Pack a universal adapter for your electronics and check voltage requirements.",
    "💊 Bring basic medications and a small first-aid kit to avoid pharmacy hunts.",
    "📷 Back up your photos daily to cloud storage in case your device is lost or stolen.",
    "🛂 Keep digital copies of important documents like passports and insurance.",
    "🍽️ Try eating where locals eat for authentic cuisine at better prices.",
]

# Safety tips by region (simplified)
safety_tips = {
    "Europe": "Watch for pickpockets in tourist areas. Keep valuables secure.",
    "Asia": "Respect local customs and dress codes. Drink bottled water in many areas.",
    "Africa": "Get necessary vaccinations before travel. Use registered tour guides.",
    "North America": "Check neighborhood safety in urban areas. Tipping is expected in service industries.",
    "South America": "Use official taxis or rideshare apps. Keep valuables out of sight.",
    "Oceania": "Apply sunscreen regularly. Be aware of wildlife and ocean safety.",
    "Middle East": "Research and respect local customs and dress codes. Stay hydrated in hot weather."
}

# Common travel phrases
travel_phrases = {
    "French": {"Hello": "Bonjour", "Thank you": "Merci", "Where is...?": "Où est...?"},
    "Spanish": {"Hello": "Hola", "Thank you": "Gracias", "Where is...?": "¿Dónde está...?"},
    "Italian": {"Hello": "Ciao", "Thank you": "Grazie", "Where is...?": "Dov'è...?"},
    "German": {"Hello": "Hallo", "Thank you": "Danke", "Where is...?": "Wo ist...?"},
    "Japanese": {"Hello": "こんにちは (Konnichiwa)", "Thank you": "ありがとう (Arigatou)", "Where is...?": "どこですか (Doko desu ka)"},
    "Mandarin": {"Hello": "你好 (Nǐ hǎo)", "Thank you": "谢谢 (Xièxiè)", "Where is...?": "在哪里 (Zài nǎlǐ)"}
}

def get_map_url(place):
    """Returns a Google Maps search URL for the place."""
    base = "https://www.google.com/maps/search/"
    return base + requests.utils.quote(place)

def get_image_url(title):
    """Returns main image URL of a Wikipedia page using Wikimedia API."""
    endpoint = f"https://en.wikipedia.org/api/rest_v1/page/summary/{requests.utils.quote(title)}"
    try:
        data = requests.get(endpoint).json()
        return data.get("thumbnail", {}).get("source", "❌ No image available.")
    except:
        return "❌ Error retrieving image."

def get_country_from_place(place_name):
    """Try to determine country from place name using geocoding."""
    try:
        geolocator = Nominatim(user_agent="smarttrip_info_finder")
        location = geolocator.geocode(place_name, exactly_one=True, language="en")

        if location and "address" in location.raw:
            address = location.raw["address"]
            country_code = address.get("country_code", "").upper()

            if country_code:
                country = pycountry.countries.get(alpha_2=country_code)
                if country:
                    return country.name
        return None
    except:
        return None

def get_country_info(country_name):
    """Get basic information about a country."""
    if not country_name:
        return None

    try:
        response = requests.get(f"https://restcountries.com/v3.1/name/{country_name}")
        if response.status_code == 200:
            data = response.json()[0]
            return {
                "capital": data.get("capital", ["Unknown"])[0],
                "population": data.get("population", "Unknown"),
                "languages": ", ".join(list(data.get("languages", {}).values())),
                "currency": list(data.get("currencies", {}).keys())[0] if data.get("currencies") else "Unknown",
                "currency_name": list(data.get("currencies", {}).values())[0].get("name", "Unknown") if data.get("currencies") else "Unknown",
                "region": data.get("region", "Unknown"),
                "subregion": data.get("subregion", "Unknown")
            }
        return None
    except:
        return None

def get_currency_rate(currency_code):
    """Get currency exchange rate against USD."""
    if currency_code == "Unknown":
        return None

    try:
        c = CurrencyRates()
        rate = c.get_rate("USD", currency_code)
        return rate
    except:
        return None

def get_weather(place_name):
    """Get current weather for a location using OpenWeatherMap API."""
    # Note: In a real app, you would use your API key here
    # Using a sample response for demonstration
    weather_data = {
        "temp": round(random.uniform(5, 35), 1),
        "conditions": random.choice(["Clear", "Cloudy", "Rainy", "Sunny", "Partly Cloudy"]),
        "humidity": random.randint(30, 90)
    }
    return weather_data

def get_best_time_to_visit(place_name):
    """Determine best time to visit based on seasonal patterns."""
    # This would normally use real data, but for demo we'll use hemisphere + random selection
    try:
        geolocator = Nominatim(user_agent="smarttrip_info_finder")
        location = geolocator.geocode(place_name)

        if location:
            latitude = location.latitude
            # Northern hemisphere
            if latitude > 0:
                return random.choice(["May to June", "September to October"])
            # Southern hemisphere
            else:
                return random.choice(["November to March", "October to April"])
        return "Spring and Fall typically offer pleasant weather and fewer crowds"
    except:
        return "Spring and Fall typically offer pleasant weather and fewer crowds"

def get_attractions(place_name):
    """Get top attractions for a destination (simplified)."""
    try:
        attractions = []
        search_query = f"top attractions in {place_name}"
        url = f"https://www.google.com/search?q={requests.utils.quote(search_query)}"

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract attraction names (simplified approach)
        for item in soup.find_all("h3")[:5]:
            if item.text and not item.text.startswith("People also ask"):
                attractions.append(item.text)

        # If we couldn't get reliable data, provide some generic attractions
        if len(attractions) < 3:
            return ["Historical Center", "Main Square/Plaza", "Local Museum", "City Park"]

        return attractions[:4]  # Return top 4 attractions
    except:
        return ["Historical Center", "Main Square/Plaza", "Local Museum", "City Park"]

def color_temp(temp):
    """Return colored temperature based on value."""
    if temp < 10:
        return Fore.CYAN + f"{temp}°C" + Style.RESET_ALL
    elif temp < 20:
        return Fore.GREEN + f"{temp}°C" + Style.RESET_ALL
    elif temp < 30:
        return Fore.YELLOW + f"{temp}°C" + Style.RESET_ALL
    else:
        return Fore.RED + f"{temp}°C" + Style.RESET_ALL

# Welcome message
print(Fore.CYAN + Style.BRIGHT + "\n" + "=" * 70)
print(Fore.CYAN + Style.BRIGHT + "🌐 Welcome to SmartTrip Info Finder 2.0")
print(Fore.LIGHTBLACK_EX + "Your ultimate AI travel companion with real-time data & practical advice!")
print(Fore.CYAN + Style.BRIGHT + "=" * 70 + "\n")

# Main loop
while True:
    print(Style.BRIGHT + Fore.YELLOW + "━" * 80)
    query = input(Fore.GREEN + "🔎 Enter a destination (or type 'exit'): ").strip()
    print(Style.BRIGHT + Fore.YELLOW + "━" * 80 + "\n")

    if query.lower() == "exit":
        print(Fore.CYAN + "\n" + "=" * 70)
        print(Style.BRIGHT + Fore.CYAN + "👋 Thank you for using SmartTrip! Safe travels! ✈️🌍")
        print(Fore.CYAN + "=" * 70 + "\n")
        break

    if not query:
        print(Fore.RED + "⚠️ Please enter a valid place name.\n")
        continue

    try:
        print(Fore.LIGHTBLUE_EX + "🔍 Searching for destination information...\n")
        results = wikipedia.search(query)
        time.sleep(1)

        if not results:
            print(Fore.RED + f"❌ No results found for '{query}'. Try something else.\n")
            continue

        # Show top suggestions
        print(Fore.MAGENTA + "📌 Top Matches:")
        for i, title in enumerate(results[:3], start=1):
            print(f"{i}. {title}")
        print()

        # Use best match
        best_match = results[0]
        print(Fore.YELLOW + f"✅ Collecting info on: {Style.BRIGHT}{best_match}\n")
        page = wikipedia.page(best_match)
        content = page.content[:3000]

        # Summarize
        print(Fore.LIGHTBLUE_EX + "🧠 Generating traveler's summary...\n")
        summary = summarizer(content, max_length=150, min_length=60, do_sample=False)[0]['summary_text']

        # Get country info
        country_name = get_country_from_place(best_match)
        country_info = None
        if country_name:
            country_info = get_country_info(country_name)

        # Get current weather (simulated)
        weather = get_weather(best_match)

        # Get attractions
        attractions = get_attractions(best_match)

        # Get best time to visit
        best_time = get_best_time_to_visit(best_match)

        # Output structured destination profile with eye-catching formatting
        print(Back.BLUE + Fore.WHITE + Style.BRIGHT + f" 🌍 DESTINATION PROFILE: {best_match.upper()} ".center(78) + Style.RESET_ALL)

        # Summary section
        print(Fore.WHITE + Style.BRIGHT + "\n📝 OVERVIEW:")
        print(Fore.LIGHTWHITE_EX + textwrap.fill(summary, width=78))

        # Country information if available
        if country_info:
            print(Fore.WHITE + Style.BRIGHT + "\n🏳️ COUNTRY INFO:")
            print(Fore.CYAN + f"  • Country: {Fore.WHITE}{country_name}")
            print(Fore.CYAN + f"  • Capital: {Fore.WHITE}{country_info['capital']}")
            print(Fore.CYAN + f"  • Region: {Fore.WHITE}{country_info['region']}, {country_info['subregion']}")
            print(Fore.CYAN + f"  • Population: {Fore.WHITE}{country_info['population']:,}")
            print(Fore.CYAN + f"  • Languages: {Fore.WHITE}{country_info['languages']}")

            # Currency information
            print(Fore.CYAN + f"  • Currency: {Fore.WHITE}{country_info['currency']} ({country_info['currency_name']})")

            rate = get_currency_rate(country_info['currency'])
            if rate:
                print(Fore.CYAN + f"  • Exchange Rate: {Fore.WHITE}1 USD = {rate:.2f} {country_info['currency']}")

        # Weather information
        print(Fore.WHITE + Style.BRIGHT + "\n🌡️ CURRENT WEATHER:")
        print(Fore.YELLOW + f"  • Temperature: {color_temp(weather['temp'])}")
        print(Fore.YELLOW + f"  • Conditions: {Fore.WHITE}{weather['conditions']}")
        print(Fore.YELLOW + f"  • Humidity: {Fore.WHITE}{weather['humidity']}%")
        print(Fore.YELLOW + f"  • Best time to visit: {Fore.WHITE}{best_time}")

        # Must-see attractions
        print(Fore.WHITE + Style.BRIGHT + "\n🏛️ TOP ATTRACTIONS:")
        for i, attraction in enumerate(attractions, 1):
            print(Fore.MAGENTA + f"  {i}. {Fore.WHITE}{attraction}")

        # Safety tip based on region
        if country_info:
            region = country_info['region']
            if region in safety_tips:
                print(Fore.WHITE + Style.BRIGHT + "\n🛡️ SAFETY TIP:")
                print(Fore.RED + f"  {safety_tips[region]}")

        # Basic phrases if available
        if country_info and country_info['languages']:
            main_language = country_info['languages'].split(',')[0].strip()
            for lang, phrases in travel_phrases.items():
                if main_language.lower() in lang.lower():
                    print(Fore.WHITE + Style.BRIGHT + f"\n💬 USEFUL PHRASES ({lang}):")
                    for phrase, translation in phrases.items():
                        print(Fore.BLUE + f"  • {phrase}: {Fore.WHITE}{translation}")
                    break

        # Show image URL
        image_url = get_image_url(best_match)
        print(Fore.WHITE + Style.BRIGHT + "\n🖼️ IMAGE PREVIEW URL:")
        print(Fore.CYAN + image_url)

        # Show Google Maps URL
        map_url = get_map_url(best_match)
        print(Fore.WHITE + Style.BRIGHT + "\n🗺️ VIEW ON GOOGLE MAPS:")
        print(Fore.GREEN + map_url)

        # Current date and local time estimate
        current_time = datetime.datetime.now()
        print(Fore.WHITE + Style.BRIGHT + "\n🕒 TRAVEL PLANNING:")
        print(Fore.YELLOW + f"  • Current Date: {Fore.WHITE}{current_time.strftime('%B %d, %Y')}")
        print(Fore.YELLOW + f"  • Ideal Trip Duration: {Fore.WHITE}5-7 days")

        # Random travel tip
        print(Fore.WHITE + Style.BRIGHT + "\n💡 TRAVELER'S TIP:")
        print(Fore.BLUE + f"  {random.choice(travel_tips)}")

        print("\n" + Style.DIM + "=" * 80 + "\n")

    except wikipedia.exceptions.DisambiguationError as e:
        print(Back.RED + Fore.WHITE + Style.BRIGHT + " ⚠️ MULTIPLE MATCHES FOUND ".center(78) + Style.RESET_ALL)
        print(Fore.RED + f"\n'{query}' could refer to several destinations. Please try one of these:")
        for i, option in enumerate(e.options[:5], 1):
            print(Fore.YELLOW + f"  {i}. {option}")
        print()

    except wikipedia.exceptions.PageError:
        print(Back.RED + Fore.WHITE + Style.BRIGHT + " ❌ DESTINATION NOT FOUND ".center(78) + Style.RESET_ALL)
        print(Fore.RED + f"\nNo information found for '{query}'. Try being more specific or check spelling.\n")

    except Exception as ex:
        print(Back.RED + Fore.WHITE + Style.BRIGHT + " 🚫 ERROR ".center(78) + Style.RESET_ALL)
        print(Fore.RED + f"\nSomething went wrong: {ex}\n")
        print(Fore.YELLOW + "Try another destination or check your internet connection.\n")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.5 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



🌐 Welcome to SmartTrip Info Finder 2.0
Your ultimate AI travel companion with real-time data & practical advice!

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔎 Enter a destination (or type 'exit'): japan
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔍 Searching for destination information...

📌 Top Matches:
1. Japan
2. Japanese
3. Anime

✅ Collecting info on: Japan

🧠 Generating traveler's summary...

                         🌍 DESTINATION PROFILE: JAPAN                         

📝 OVERVIEW:
Japan is an island country in East Asia. It consists of four major islands and
thousands of smaller islands, covering 377,975 square kilometers (145,937 sq
mi) Japan has a population of over 123 million as of 2025, making it the
eleventh-most populous country. The capital of Japan and its largest city is
Tokyo; the Greater Tokyo Area is the largest metropolitan area in the world.

🌡️ CURRENT WEATHER:
  • Temperature: 18.3°C
 